##### DNN기반 회귀 모델 구현
- 데이터셋 : iris.csv
- 피쳐/속성 : 3개 Sepal_Length, Sepal_Width, Petal_Length
- 타겟/라벨 : 1개 Petal_Width
- 학습 방법 : 지도학습 > 회귀
- 알고리즘  : 인공신경망 (ANN) -> MLP,DNN  : 은닉층이 많은 구성
- 프레임워크: Pytorch

***
- 모니터링 
    * 기준설정 : 검증데이터셋의 loss와 score
    * 평가 : 학습데이터셋의 loss와 score와 비교해서 학습 중단 여부 결정
    * 선택 : 현재까지 진행된 모델의 파라미터(가중치와, 절편) 저장 여부 또는 모델 전체 저장


[1] 모듈 로딩 및 데이터 준비 <hr>

In [14]:
# 모듈 로딩
# -Model 관련
import torch
import torch.nn as nn
import torch.nn.functional as F # 변수명과 헷갈릴까봐 소문자는 사용 X

from torch.utils.data import Dataset, DataLoader
import torch.optim as optim

from torchmetrics.regression import R2Score, MeanSquaredError

from torchinfo import summary

# - Data 및 시각화 관련
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import *
from sklearn.model_selection import train_test_split 

In [15]:
# 활용 패키지 버전 체크 ==> 사용자 정의 함수로 구현하기
print(f'Pytorch v.{torch.__version__}')
print(f'Pandas v.{pd.__version__}')

Pytorch v.2.4.1
Pandas v.2.0.3


In [16]:
### 데이터 로딩
DATA_FILE = r'C:\Users\KDP-27\Desktop\KDT6\MachineLearning\data\iris.csv'

### CSV >>> DataFrame
irisDF = pd.read_csv(DATA_FILE, usecols=[0,1,2,3])

### 확인
irisDF.head(1)

,sepal.length,sepal.width,petal.length,petal.width
0,5.1,3.5,1.4,0.2


[2] 모델 클래스 설계 및 정의 <hr>
- 클래스 목적 : iris 데이터를 학습 및 추론 목적
- 클래스 이름 : IrisRegModel
- 부모 클래스 : nn.Module
- 매 개 변 수 : 층별 입/출력 개수 고정이므로 필요 없음!
- 속 성 필 드 : 
- 기능 / 역할 
    * _ _ _init_ _ _() : 모델 구조 설정
    * forward() : 순방향 학습 <= 오버라이딩(overriding, 상속관계에서만 가능)
- 클래스 구조 
    * 입력층 : 입력  3개(피쳐 수)  출력 10개(퍼셉트론/뉴런 10개 존재)
    * 은닉층 : 입력 10개           출력 30개(퍼셉트론/뉴런 30개 존재)   
    * 출력층 : 입력 30개           출력  1개(타겟-너비값)

- 활성화함수
    * 클래스 형태 ==> nn.MESLoss, nn.ReLU ==> _ _ _init_ _ _() 메서드에서 사용
    * 함수 형태 ==> torch.nn.functional 아래에 ==> forward() 메서드에서 사용

In [17]:
class IrisRegModel(nn.Module):

    # 모델 구조 구성 및 인스턴스 생성 메서드
    def __init__(self):
        super().__init__()

        self.in_layer = nn.Linear(3,10)
        self.hd_layer = nn.Linear(10,30)
        self.out_layer = nn.Linear(30,1)

    # 순방향 학습 진행 메서드
    def forward(self, input_data):

        # - 입력층
        y = self.in_layer(input_data)  # f11w11+f12w12+f13w13+b, ... , f101w101+f102w102+f103w103+b
        y = F.relu(y)                  # relu => y값의 범위 : 0 <= y (sigmoid면 y값은 0에서 1사이가 됨)

        # - 은닉층 : 10개의 숫자 값 (>=0)
        y = self.hd_layer(y)            # f21w21+f22w22+...+f210w210+b, ... , f230w101...+f230w210+b
        y = F.relu(y)                   # relu => y값의 범위 : 0 <= y

        # - 출력층 : 30개의 숫자 값 (>=0) => 회귀이므로 바로 반환(return)
        return self.out_layer(y)               # f31W31+...f330W330+b


In [18]:

# 모델 인스턴스 생성
model = IrisRegModel()

print(model)

IrisRegModel(
  (in_layer): Linear(in_features=3, out_features=10, bias=True)
  (hd_layer): Linear(in_features=10, out_features=30, bias=True)
  (out_layer): Linear(in_features=30, out_features=1, bias=True)
)


In [19]:
# 모델 사용 메모리 정보 확인
summary(model, input_size=(100000,3)) #iris 꽃 데이터 10개를 넣겠다 그때의 메모리를 계산해줌
# 그냥 계산만 해줌

Layer (type:depth-idx)                   Output Shape              Param #
IrisRegModel                             [100000, 1]               --
├─Linear: 1-1                            [100000, 10]              40
├─Linear: 1-2                            [100000, 30]              330
├─Linear: 1-3                            [100000, 1]               31
Total params: 401
Trainable params: 401
Non-trainable params: 0
Total mult-adds (M): 40.10
Input size (MB): 1.20
Forward/backward pass size (MB): 32.80
Params size (MB): 0.00
Estimated Total Size (MB): 34.00

[3] 데이터셋 클래스 설계 및 정의 <hr>
- 데이터_셋 : iris.csv
- 피쳐_개수 : 3개
- 타겟_개수 : 1개
- 클래스이름 : IrisDataset
- 부모클래스 : utils.data.Dataset
- 속성__필드 : featureDF, targetDF, n_rows, n_features
- 필수메서드 : 
    - _ _init_ _(self) : 데이터셋 저장 및 전처리, 개발자가 필요한 속성 설정
    - _ _len_ _(self) : 데이터의 개수 반환
    - _ _getitem_ _(self, index) : 특정 인덱스의 피쳐와 타겟 반환

In [20]:
class IrisDataset(Dataset):
    
    def __init__(self, featureDF, targetDF):
        super().__init__()

        self.featureDF=featureDF
        self.targetDF=targetDF
        self.n_rows=featureDF.shape[0]
        self.n_features=featureDF.shape[1]

    def __len__(self):
        return self.n_rows

    def __getitem__(self, index):
        # 텐서화
        featureTS = torch.FloatTensor(self.featureDF.iloc[index].values) # 시리즈를 array로 만들기위해 values함 
        targetTS = torch.FloatTensor(self.targetDF.iloc[index].values)

        # 피쳐와 타겟 반환
        return featureTS, targetTS

In [21]:
## 데이터셋 인스턴스 생성

# - DataFrame에서 피쳐와 타겟 추출
featureDF = irisDF[irisDF.columns[:-1]]     # 2D (150, 3)
targetDF = irisDF[irisDF.columns[-1:]]      # 2D (150, 1)

# - 커스텀데이터셋 인스턴스 생성
irisDS = IrisDataset(featureDF, targetDF)


[4] 학습 준비
- 학습횟수 : EPOCH <- 처음부터 끝까지 공부하는 단위
- 배치크기 : BATCH_SIZE <- 한번에 학습할 데이터셋 양
- 위치지정 : DEVICE     <- 텐서 저장 및 실행 위치 (GPU/CPU)
-  학습율  : LR 가중치와 절편 업데이트 시 경사하강법으로 업데이트 간격 설정 0.001 ~ 0.1 사이 (하이퍼파라미터)

In [22]:
### 학습 진행 관련 설정
EPOCH = 1
BATCH_SIZE = 10 # 전체 데이터 수 확인후 지정 
BATCH_CNT = irisDF.shape[0]/BATCH_SIZE
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
LR = 0.001  # 하이퍼파라미터, 값이 적을수록 촘촘하게 내려감

- 인스턴스/객체 : 모델, 데이터셋, 최적화, (,손실함수 , 성능지표) 

In [23]:
#  모델 인스턴스
model=IrisRegModel()

# 데이터셋 인스턴스
X_train, X_test, y_train, y_test = train_test_split(featureDF,targetDF, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, random_state=1)
print(f'{X_train.shape} {X_test.shape} {X_val.shape}')
print(f'{y_train.shape} {y_test.shape} {y_val.shape}')
print(f'{type(X_train)} {type(X_test)} {type(X_val)}')

trainDS=IrisDataset(X_train,y_train)
valDS=IrisDataset(X_val,y_val)
testDS=IrisDataset(X_test,y_test)


#irisDF=IrisDataset(featureDF,targetDF)

# 데이터로더 인스턴스
trainDL=DataLoader(trainDS, batch_size=BATCH_SIZE)

(84, 3) (38, 3) (28, 3)
(84, 1) (38, 1) (28, 1)
<class 'pandas.core.frame.DataFrame'> <class 'pandas.core.frame.DataFrame'> <class 'pandas.core.frame.DataFrame'>


In [24]:
## [테스트] 데이터로더와 데이터셋 체크
for feature, target in trainDL:
     #print(feature,target) => 배치사이즈 만큼 출력
     print(feature.shape, target.shape)

torch.Size([10, 3]) torch.Size([10, 1])
torch.Size([10, 3]) torch.Size([10, 1])
torch.Size([10, 3]) torch.Size([10, 1])
torch.Size([10, 3]) torch.Size([10, 1])
torch.Size([10, 3]) torch.Size([10, 1])
torch.Size([10, 3]) torch.Size([10, 1])
torch.Size([10, 3]) torch.Size([10, 1])
torch.Size([10, 3]) torch.Size([10, 1])
torch.Size([4, 3]) torch.Size([4, 1])


In [25]:
# 최적화 인스턴스 w,b 텐서 즉, model.parameter() 전달
optimizer=optim.Adam(model.parameters(), lr=LR)

# 손실함수 인스턴스 => 회귀, MSE, MAE, RMSE...
reqloss = nn.MSELoss()

[5] 학습진행

In [26]:
## 학습의 효과 확인 손실값과 성능평가값 저장 필요
LOSS_HISTORY, SCORE_HISTORY =[[],[]],[[],[]]
CNT=irisDS.n_rows/BATCH_SIZE
print(f'CNT => {CNT}')

## 학습 모니터링, 스케쥴링 설정
# => LOSS_HISTORY, SCORE_HISTORY 활용
# => 임계기준 : 10번

BREAK_CNT=0
LIMIT_VALUE=10



for epoch in range(EPOCH):

    # 학습 모드로 모델 설정
    model.train()

    # 배치 크기 만큼 데이터 로딩해서 학습 진행
    loss_total, score_total=0,0

    for featureTS, targetTS in trainDL:
        # 학습 진행 
        pre_y=model(featureTS)

        # 손실 계산
        loss=reqloss(pre_y,targetTS)
        loss_total += loss.item()

        # 성능평가 계산
        score= R2Score()(pre_y,targetTS)
        score_total += score.item()

        # 최적화 진행
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # 에포트 당 겅증기능
    # 모델 검증 모드 설정
    model.eval()
    with torch.no_grad():

        # 검증 데이터셋
        val_featureTS=torch.FloatTensor(valDS.featureDF.values)
        val_targetTS=torch.FloatTensor(valDS.targetDF.values)

        #추론/평가
        pre_val=model(val_featureTS)

        #손실
        loss_val=reqloss(pre_val,val_targetTS)
        
        # 성능평가
        score_val=R2Score()(pre_val, val_targetTS)



    # 에포크 당 손실값과 성능평가값 저장
    LOSS_HISTORY[0].append(loss_total/CNT)
    SCORE_HISTORY[0].append(score_total/CNT)

    LOSS_HISTORY.append(loss_total/BATCH_SIZE)
    SCORE_HISTORY.append(score_total/BATCH_SIZE)

    # 학습 진행 모니터링 / 스케쥴링 - 검증 DS기준
    #LOSS  기준
    if len(LOSS_HISTORY[1])>=2:
        if LOSS_HISTORY[1][-1] >= LOSS_HISTORY[1][-1] :BREAK_CNT += 1
    
    # # SCORE 기준    
    #     if len(SCORE_HISTORY[1])>=2:
    #         if SCORE_HISTORY[1][-1] <= SCORE_HISTORY[1][-1] :BREAK_CNT += 1

    # 학습 중단 여부 설정
    if BREAK_CNT>=LIMIT_VALUE:
        print("성능 및 손실 개선이 없어 학습 중단")
        break

CNT => 15.0


In [35]:
val_featrueTS=torch.FloatTensor(valDS.featureDF.values)
val_targetTS=torch.FloatTensor(valDS.targetDF.values)

val_featrueTS.shape, val_targetTS.shape

(torch.Size([28, 3]), torch.Size([28, 1]))

In [30]:
print(f'LOSS_HISTORY => {LOSS_HISTORY}')
print(f'SCORE_HISTORY => {SCORE_HISTORY}')

LOSS_HISTORY => [[0.5049956977367401], [], 0.7574935466051101]
SCORE_HISTORY => [[-0.49732359250386554], [], -0.7459853887557983]


- 모델 저장방법
---
- 방법1: 모델 파라미터만 저장
- 방법2: 모델 설계 구조 및 파라미터까지 모두 저장

In [31]:
### 학습된 모델 파라미터 값 확인

model.state_dict()

OrderedDict([('in_layer.weight',
              tensor([[-0.4622,  0.5626, -0.0374],
                      [ 0.0945, -0.5760, -0.1194],
                      [-0.3465, -0.0381,  0.5463],
                      [ 0.4580, -0.0075,  0.5410],
                      [-0.5405,  0.2267, -0.1517],
                      [ 0.2113,  0.0650, -0.2891],
                      [ 0.1053, -0.4307, -0.5469],
                      [-0.1051,  0.1687, -0.5473],
                      [-0.4533,  0.3912, -0.5700],
                      [-0.0834,  0.2591,  0.2629]])),
             ('in_layer.bias',
              tensor([-0.2283,  0.4185,  0.2143,  0.2669,  0.1360,  0.1195, -0.3143, -0.4293,
                      -0.4598,  0.1130])),
             ('hd_layer.weight',
              tensor([[ 0.0523,  0.3152,  0.2978, -0.0245, -0.0794, -0.2463,  0.2402, -0.2035,
                        0.2161, -0.2239],
                      [-0.2088, -0.1662,  0.0669, -0.2212, -0.0390,  0.2304, -0.0756, -0.3052,
                     

In [32]:
### models 폴더 아래 프로젝트 폴더 아래 모델 파일 저장
import os

# 저장 경로
SAVE_PATH='../models/iris'

# 저장 파일명
SAVE_FILE='model_train_wbs.pth'

In [33]:
# 경로상 폴더 존재 여부 체크
if not os.path.exists(SAVE_PATH):
    os.makedirs(SAVE_PATH)          #폴더/폴더/... 하위 폴더까지 생성


In [34]:
# 모델 저장
torch.save(model.state_dict(),SAVE_PATH+SAVE_FILE)

In [38]:
# 모델 즉, 가중치와 절편 로딩
# [1] 가중치와 절편 객체로 로딩
# [2] 모델의 state_dict 속성에 저장

# 읽기
wbTS=torch.load(SAVE_PATH+SAVE_FILE,weights_only=True)
print(type(wbTS))

<class 'collections.OrderedDict'>


In [40]:
# 모델 인스턴스에 저장
model2 = IrisRegModel()     #층마다 w,b 초기화

model2.load_state_dict(wbTS)

<All keys matched successfully>

In [41]:
print(model2)

IrisRegModel(
  (in_layer): Linear(in_features=3, out_features=10, bias=True)
  (hd_layer): Linear(in_features=10, out_features=30, bias=True)
  (out_layer): Linear(in_features=30, out_features=1, bias=True)
)
